## 1. Importar Librerías

Importamos las librerías del **Agent Framework**:
- `executor`: Decorador para definir executors
- `WorkflowBuilder`: Constructor de workflows
- `WorkflowContext`: Contexto de ejecución
- `WorkflowOutputEvent`: Evento de salida final

In [ ]:
import asyncio
from typing_extensions import Never
from agent_framework import WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor

## 2. Definir el Primer Executor: Mayúsculas

### Executor `to_upper_case`
- **Input**: String de texto
- **Proceso**: Convierte a mayúsculas con `.upper()`
- **Output**: Envía el resultado al siguiente executor con `ctx.send_message()`

### Parámetros importantes:
- `id`: Identificador único del executor
- `ctx: WorkflowContext[str]`: Contexto que envía strings al siguiente paso

In [ ]:
@executor(id="upper_case_executor")
async def to_upper_case(text: str, ctx: WorkflowContext[str]) -> None:
    """Transform the input to uppercase and forward it to the next step."""
    result = text.upper()

    # Send the intermediate result to the next executor
    await ctx.send_message(result)

print("✅ Executor 'to_upper_case' definido")

## 3. Definir el Segundo Executor: Invertir Texto

### Executor `reverse_text`
- **Input**: String en mayúsculas del executor anterior
- **Proceso**: Invierte el texto con slicing `[::-1]`
- **Output**: Emite el resultado final con `ctx.yield_output()`

### Diferencia clave:
- `send_message()`: Envía a otro executor
- `yield_output()`: Emite el resultado final del workflow

### Type hint especial:
- `WorkflowContext[Never, str]`: No envía mensajes (`Never`), solo produce salida (`str`)

In [ ]:
@executor(id="reverse_text_executor")
async def reverse_text(text: str, ctx: WorkflowContext[Never, str]) -> None:
    """Reverse the input and yield the workflow output."""
    result = text[::-1]

    # Yield the final output for this workflow run
    await ctx.yield_output(result)

print("✅ Executor 'reverse_text' definido")

## 4. Construir el Workflow

Usamos `WorkflowBuilder` para conectar los executors:

### Arquitectura:
```
Input → to_upper_case → reverse_text → Output
```

### Métodos del builder:
1. `add_edge(from, to)`: Conecta dos executors
2. `set_start_executor(executor)`: Define el primer executor
3. `build()`: Compila el workflow

In [ ]:
workflow = (
    WorkflowBuilder()
    .add_edge(to_upper_case, reverse_text)
    .set_start_executor(to_upper_case)
    .build()
)

print("✅ Workflow construido")
print("Flujo: Input → to_upper_case → reverse_text → Output")

## 5. Ejecutar el Workflow

### Entrada de ejemplo:
- Input: `"hello world"`

### Procesamiento esperado:
1. `to_upper_case`: "hello world" → "HELLO WORLD"
2. `reverse_text`: "HELLO WORLD" → "DLROW OLLEH"

### Streaming de eventos:
- Usamos `run_stream()` para obtener eventos en tiempo real
- El `WorkflowOutputEvent` contiene el resultado final

In [ ]:
async def main():
    """Ejecuta el workflow con un texto de ejemplo"""
    input_text = "hello world"
    
    print(f"\n{'='*60}")
    print("EJECUCIÓN DEL WORKFLOW")
    print(f"{'='*60}")
    print(f"\nInput: '{input_text}'")
    print(f"\nProcesando...\n")
    
    # Run the workflow and stream events
    async for event in workflow.run_stream(input_text):
        print(f"Event: {event}")
        if isinstance(event, WorkflowOutputEvent):
            print(f"\n{'='*60}")
            print("RESULTADO FINAL")
            print(f"{'='*60}")
            print(f"Output: '{event.data}'")
            print(f"\n✅ Workflow completado exitosamente")

print("✅ Función main() definida")
print("\n⚠️ Ejecuta la siguiente celda para correr el workflow")

In [ ]:
# Ejecutar el workflow
await main()

## 6. Probar con Diferentes Inputs

Vamos a probar el workflow con diferentes textos:

In [ ]:
# Test con diferentes inputs
test_inputs = [
    "Python",
    "Agent Framework",
    "Azure AI"
]

async def test_multiple_inputs():
    """Prueba el workflow con múltiples inputs"""
    print(f"\n{'='*60}")
    print("PRUEBAS CON MÚLTIPLES INPUTS")
    print(f"{'='*60}\n")
    
    for test_input in test_inputs:
        print(f"Input: '{test_input}'")
        
        # Ejecutar workflow
        async for event in workflow.run_stream(test_input):
            if isinstance(event, WorkflowOutputEvent):
                print(f"Output: '{event.data}'")
        print()

await test_multiple_inputs()

## Conclusión

Este ejemplo demuestra:

### 1. **Executors Básicos**
- Funciones async decoradas con `@executor`
- Procesamiento simple de datos
- Paso de información entre executors

### 2. **WorkflowContext**
- `send_message()`: Envía datos al siguiente executor
- `yield_output()`: Emite el resultado final
- Type hints para control de flujo

### 3. **WorkflowBuilder**
- API fluent para construir workflows
- Definición clara de edges (conexiones)
- Punto de inicio configurable

### 4. **Event Streaming**
- `run_stream()`: Streaming asíncrono de eventos
- `WorkflowOutputEvent`: Evento de resultado final
- Procesamiento iterativo de eventos

### Aplicaciones Prácticas:

✅ **Pipelines de datos**: ETL (Extract, Transform, Load)

✅ **Procesamiento de texto**: Limpieza, normalización, análisis

✅ **Validación**: Múltiples pasos de validación secuencial

✅ **Transformación**: Cadenas de transformaciones de datos

### Comparación con Agentes:

| Aspecto | Executors Simples | Agentes LLM |
|---------|------------------|-------------|
| Velocidad | ⚡ Muy rápido | 🐌 Más lento |
| Costo | 💰 Gratis | 💰💰 Con costo |
| Lógica | 🎯 Determinística | 🎲 Probabilística |
| Casos de uso | Transformaciones simples | Decisiones complejas |

### Extensiones posibles:

🔹 Agregar más executors en la cadena

🔹 Implementar validaciones entre pasos

🔹 Manejar errores y reintentos

🔹 Combinar con agentes LLM para decisiones inteligentes